In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("lab4.ipynb")

---

<h1><center>SDSE Lab 4 <br><br> Linear regression and Feature selection </center></h1>

---

In this lab we will use linear regression to predict cancer mortality rates based on data obtained from the American Community Survey of the [U.S. Census Bureau](https://www.census.gov/). The lab has four parts. In part 1 you will load the data and do basic manipulations using [pandas](https://pandas.pydata.org/docs/index.html). Pandas is a Python package that specializes in tabular data. It is widely used in data science and machine learning since the data in these fields are usually structured as a table. Pandas is a very powerful library that is well worth investing some time in. [Here](https://pandas.pydata.org/docs/getting_started/index.html#getting-started) and [here](https://pandas.pydata.org/docs/user_guide/index.html) are resources to learn more.

In part 2 you will perform linear regression on the full feature set. In part 3 you will compute confidence intervals on the paramters from part 2. Finally, in part 4 you will run the forward and backward stepwise feature selection algorithms and estimate the performance of the resulting model using a test dataset.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as stats
from resources.hashutils import *

# Part 1:  Loading and cleaning the data

## 1.1 Load the data into a pandas DataFrame

Use [pd.read_csv](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) to load the data from `cancerdata.csv`.

You can obtain information about the data using these DataFrame methods and attributes:
+ [`data0.head()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html): displays the first 5 rows of the DataFrame.
+ [`data0.tail()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.tail.html): displays the last 5 rows of the DataFrame.
+ [`data0.shape`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.shape.html): a tuple with the number of rows and of columns in the DataFrame.
+ [`data0.columns`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.columns.html): the column headers.
+ [`data0.index`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.index.html): a unique identifier for each row.

In [ ]:
data0 = ...  # TODO

## 1.2 Inspect columns

Run `data0.info()` and note:
 a) which inputs are non-numerical (Dtype=object), and
 b) which inputs have null entries (Non-Null Count<3047).

Store the names ('Column' entry) of the non-numerical inputs in a [set](https://www.w3schools.com/python/python_sets.asp) called `non_numerical_inputs`. Store the names of inputs with null entries in a set called `null_entry_inputs`.

**Note**: If not all of the rows of `data0.info()` are displayed, you'll probably have this message at the bottom:

*``Output is truncated. View as a scrollable element or open in a text editor. Adjust cell output settings...''*

In [ ]:
data0.info()
non_numerical_inputs = {...,...}      # TODO
null_entry_inputs = {...,...,...}     # TODO

In [ ]:
grader.check("q1p2")

## 1.3 Discard non-numerical columns

Remove the two columns with non-numeric data.

Hints:
+ `data0.dtypes` lists the data types for each column.
+ You can construct a boolean mask for non-numeric columns with `data0.dtypes=='object'`.
+ Use that mask to index `data.columns`
+ Use [`data0.drop`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html) to remove the selected columns.
+ Save the result as `data1`

In [ ]:
ind = ...               # TODO
drop_cols = ...         # TODO
data1 = data0.drop(...)       # TODO

In [ ]:
grader.check("q1p3")

## 1.4 Discard columns where more than 10% of values are nan

Hints:
+ [`.dropna`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html)
+ Use `axis=1` to drop columns (as opposed to `axis=1` for rows.
+ Use the `thresh` argument. The condition for dropping a column is that it has less than `round(0.9*data1.shape[0])` non-nans.
+ Save the result as `data2`

In [ ]:
thresh = ...                # TODO
data2 = data1.dropna(...)     # TODO

In [ ]:
grader.check("q1p4")

## 1.5 Drop all rows that contain one or more nans.

Save the result as `data3`.

Hint: You can again use `dropna` for this.

In [ ]:
data3 = ...      # TODO

In [ ]:
grader.check("q1p5")

## 1.6 Inspect correlations

Next we'll look at the sample correlation coefficients between each of the inputs and the output (a.k.a. target variable) `target_deathrate`. This is a quick way to check which of the inputs may be most useful to include in a model. Correlations only provide an initial guess, however. Remember that the correlation coefficient only measures the linear relationship between variables. That's perfect when the model is linear (as in this lab activity), but less useful for nonlinear models.

1) Use `data3.corr()` to build the correlations matrix.
2) Inspect the column (or row) corresponding to `target_deathrate`.
3) Rank (i.e. sort) the inputs from most to least correlated with the output. This ranking is in terms of the absolute value of the sample correlation coefficient.
4) Save the top 5 correlated inputs to `top_5_corr`. `top_5_corr` should be a numpy array with shape `(5,)`.

Hints:
+ [`abs`](https://pandas.pydata.org/docs/reference/api/pandas.Series.abs.html)
+ [`sort_values`](https://pandas.pydata.org/docs/reference/api/pandas.Series.sort_values.html)
+ [`to_numpy`](https://pandas.pydata.org/docs/reference/api/pandas.Series.to_numpy.html)

In [ ]:
# correlation matrix
C = ...         # TODO

# vector correlations between the inputs and target_deathrate
corr_target = ...         # TODO

# sorted corr_target_sort
corr_target_sort = ...         # TODO

# top 5 correlations with target_deathrate
top_5_sort = ...         # TODO

In [ ]:
grader.check("q1p6")

## 1.7 Scatter plot

Make a scatter plot of the data with the most correlated input along the x axis, and the target along the y axis.

Hint: You can use the plotting function attached to the DataFrame: [data3.plot(kind='scatter',x=..., y=...)](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.html)

In [ ]:
data3.plot(kind='scatter',x=...,y=...)   # TODO

---

# Part 2: Linear regression 

## 2.1 Extract `X` and `Y` from `data3`

The next cell extracts the `X` and `Y` matrices from `data3` and constructs a list of `inputs`. Define the number of samples `N` and the total number of inputs `D`.

In [ ]:
X = data3.drop(columns='target_deathrate').values
inputs = data3.columns.values
inputs = inputs[inputs!='target_deathrate']
Y = data3['target_deathrate'].values

N = ...
D = ...

In [ ]:
grader.check("q2p1")

## 2.2 Center the inputs

\begin{align*}
\hat\mu_X &= \frac{1}{N} \mathbf{1}^T_N \mathbf{X} \\
\mathbf{X}^c&=\mathbf{X}-\mathbf{1}_N\hat\mu_X
\end{align*}

Compute the column-wise means `muhatX` and subtract them from `X` to obtain `Xc`. 

Hint: 
+ Use `X.mean(axis=...)`. Should it be `axis=0` or `axis=1`? 
+ The formula for `Xc` above has $\mathbf{1}_N\hat\mu_X$. The broadcasting rules of numpy make multiplying $\hat\mu_X$ by $\mathbf{1}_N$ unnecessary.

Check that the column-wise means of `Xc` equal zero (to machine precision). 

In [ ]:
muhatX = ...    # TODO
Xc = ...        # TODO

In [ ]:
grader.check("q2p2")

## 2.3 Center the outputs

Compute `muhatY` and the centered outputs `Yc` according to

\begin{align*}
\hat\mu_Y &= \frac{1}{N} \mathbf{1}^T_N \mathbf{Y} \\
\mathbf{Y}^c&=\mathbf{Y}-\mathbf{1}_N\hat\mu_Y
\end{align*}


In [ ]:
muhatY = ...  # TODO
Yc = ... # TODO

In [ ]:
grader.check("q2p3")

## 2.4 Compute the input covariance martrix.


$$\Sigma^{2}_X=\frac{1}{N-1}(\mathbf{X}^c)^T \mathbf{X}^c$$

Hints:
+ NumPy's @ operator and matrix transposition (`Xc.T`). See [this](https://numpy.org/devdocs/user/numpy-for-matlab-users.html) summary of the key differences between numpy and Matlab.


In [ ]:
Sigma2X = ...   # TODO

In [ ]:
grader.check("q2p4")

# Compute Input/Output covariance matrix

$$\Sigma_{XY}=\frac{1}{N-1}(\mathbf{X}^c)^T \mathbf{Y}^c$$

In [ ]:
SigmaXY = ...   # TODO

In [ ]:
grader.check("q2p5")

## 2.6 Compute the least squares estimates of the parameters

\begin{align*}
\underline{\hat\theta}_1 &= (\Sigma^{2}_X)^{-1}\Sigma_{XY} \\
\hat\theta_0 &= \hat\mu_Y - \hat\mu_X \underline{\hat\theta}_1
\end{align*}


Hint:
+ [np.linalg.inv](https://numpy.org/doc/stable/reference/generated/numpy.linalg.inv.html)


In [ ]:
theta1hat = ...  # TODO
theta0hat = ...  # TODO

In [ ]:
grader.check("q2p6")

## 2.7 Compute predictions for each of the training samples

$$\mathbf{\hat{Y}} = \mathbf{1}_N\hat\theta_0 + \mathbf{X}\underline{\hat\theta}_1$$


In [ ]:
Yhat = ...  # TODO

In [ ]:
grader.check("q2p7")

## 2.8 Model performance

Compute the coefficient of determination $R^2$ for this model on the training data.

$$ R^2 = 1 - \frac{\sum_{i=1}^N (y_i-\hat{y}_i)^2}{\sum_{i=1}^N (y_i-\hat\mu_Y)^2}$$

In [ ]:
R2 = ...    # TODO

In [ ]:
grader.check("q2p8")

---

# Part 3: Parameter uncertainty

## 3.1 Estimate the variance of the output

$$\hat\sigma^2 = \frac{1}{N-D-1} \sum_{i=1}^{N}(y_i-\hat{y}_i)^2$$

In [ ]:
sigmahat2 = ...

In [ ]:
grader.check("q3p1")

## 3.2 Compute the variances of the slope parameters

These are the diagonal entries of 

$$Var\left[ \underline{\widehat{\Theta}}_1 \right]\;=\;   \sigma^2  \left((\mathbf{X}^c)^T \mathbf{X}^c \right)^{-1}  $$

The variance of $\hat\theta_d$ is the $(d\!-\!1)$'th diagonal entry of $\hat\sigma^2 (\Sigma^{2}_X)^{-1}$.

`var_thetahat` should be a 1D array with the variances of each of the slope parameters. 

**Hint**: [`np.diag`](https://numpy.org/doc/stable/reference/generated/numpy.diag.html)

In [ ]:
var_thetahat = ... # TODO

In [ ]:
grader.check("q3p2")

## 3.3 Compute the radiuses of the confidence intervals

Having the variances of each of the slope parameters in `var_thetahat`, and assuming that the output measurement noise is Gaussian, we can compute the radius of the confidence interval for the $d$'th slope parameter with
$$\rho_d =\sqrt{v_d} \left| F^{-1}_{\mathcal{N}}\left( \frac{1-\gamma}{2}\right)\right| $$
Here $v_d$ is the variance of the $d$'th slope parameter, i.e. `var_thetahat[d]`.

Compute the radiuses of the 95\% confidence intervals for the slope parameters. Store these in the array `rho`.

In [ ]:
gamma = 0.95
rho = ... # TODO

In [ ]:
grader.check("q3p3")

## 3.4 Tag as "significant" those parameters whose confidence interval does not include zero.

Create a 1D NumPy boolean array of the same size as `theta1hat` called `significant`. The $d$'th entry of `significant` should be `True` if the 95\% confidence interval for the corresponding slope parameter **does not** include 0, and `False` otherwise. In other words, an input is considered significant if its slope parameter is non-zero with 95\% confidence. 

In [ ]:
significant = ...

In [ ]:
grader.check("q3p4")

## 3.5 Parameters table (done already)
Make a DataFrame with one row for each input. The index of the table should be the input names. The columns should be:
+ `slope`: the estimates of the slope parameter associated with the input. 
+ `slope stddev`: the standard deviation of the slope parameter.
+ `significant`: whether the input is significant according to part 3.4.

In [ ]:
params_table = pd.DataFrame(index=all_inputs,
             data={'slope':theta1hat,
                   'slope stddev':np.sqrt(var_thetahat),
                   'significant':significant})

params_table

## 3.6 Build an array of significant inputs

Extract the names of the significant inputs from `params_table` using the `significant` array from part 3.4.

Store these significant input names as `significant_inputs`. 

`significant_inputs` should be a NumPy array with shape `(15,)`.

Here's one way you can do this that doesn't require a "for" loop:
1. Use the `significant` column to select the rows of the table corresponding to significant inputs. 
2. Use `.index` to obtain the names of the inputs for those rows. 
3. Use `.to_numpy()` to convert the result to a NumPy array.

In [ ]:
...
significant_inputs = ...  # TODO

In [ ]:
grader.check("q3p6")

## 3.7 Create a new table with significant inputs only (done already)

This table is called `data` and the target variable is now called `Y`.

In [ ]:
data = data3[significant_inputs].copy()
data['Y'] = data3['target_deathrate']
data

---

# Part 4: Feature subset selection 

## 4.1 Split `data` into training, validation, and testing datasets (done already)

We will use 70% of the data for training, 15% for validation, and 15% for testing.

1. Define `Dtrain` as the first `Ntrain` rows of `data`.
2. Define `Dvalidate` as the next `Nvalidate` rows of `data`.
3. Define `Dtest` as the last `Ntest` rows of `data`.

Here we use pandas' [iloc](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.iloc.html) method for selecting the three datasets.

In [ ]:
Ntrain = round(0.7*N)
Nvalidate = round(0.15*N)
Ntest = N - Ntrain - Nvalidate
Ntrain, Nvalidate, Ntest

Dtrain = data.iloc[:Ntrain,:]
Dvalidate = data.iloc[Ntrain:Ntrain+Nvalidate,:]
Dtest = data.iloc[Ntrain+Nvalidate:,:]

## 4.2 Linear regression training function

Create a function called `train` that receives a list of features `S` and a dataset `Dtrain` and does the following:
1. Selects the features `S` from `Dtrain` and stores them in `X`. (done already)
2. Selects the target values from `Dtrain` and stores them in `Y`. (done already)
3. Performs the linear regression calculations from parts 2.2, 2.3, and 2.4 (copy your code from those parts into the `train` method)
4. Returns the estimated parameters `theta0hat` and `theta1hat`.

In [ ]:
def train(S, Dtrain):

    X = Dtrain[list(S)].values
    Y = Dtrain['Y'].values

    # 2.2 Center the inputs 
    muhatX = ...    # TODO
    Xc = ...    # TODO

    # 2.3 Center the outputs 
    muhatY = ...    # TODO
    Yc = ...    # TODO

    # 2.4 Compute the inverse input covariance matrix
    invCovX = ...    # TODO

    # 2.5 Compute the least squares estimates of the parameters
    theta1hat = ...    # TODO
    theta0hat = ...    # TODO

    return theta0hat, theta1hat

In [ ]:
# Use this cell to test your code

theta0hat, theta1hat = train(['incidencerate','birthrate'], Dtrain)

In [ ]:
grader.check("q4p2")

## 4.3 Model assessment function

Create a function called `assess` that receives the linear regression parameters `theta0hat` and `theta1hat`, their corresponding feature names `S`, and a dataset `D`, which may be the validation, the training, or the testing dataset.

The function should evaluate the mean squared error (MSE) of the model using this data.

The steps are:
1. Select the features `S` from `D` and stores them in `X`. (done already)
2. Select the target values from `D` and stores them in `Y`. (done already)
3. Compute `Yhat`, as in part 2.6.
4. Evaluate MSE:

$$MSE = \frac{1}{N}\sum_{i=1}^{N} ( y_i-\hat{y}_i)^2$$

In [ ]:
def assess( S, theta0hat, theta1hat, D):

    X = D[list(S)].values
    Y = D['Y'].values

    # 2.6 Compute predictions for each of the samples
    Yhat = ... # TODO

    # 2.7 Model performance
    MSE = ... # TODO
    
    return MSE

In [ ]:
# Use this cell to test your code

theta0hat, theta1hat = train(['incidencerate','birthrate'], Dtrain)
mse = assess(['incidencerate','birthrate'], theta0hat, theta1hat, Dtrain)

In [ ]:
grader.check("q4p3")

## 4.4 Forward stepwise selection

Below is a method that implements the forward stepwise feature selection algorithm that was described by your GSI in lab. 

This part has no deliverables.

In [ ]:
def forward_stepwise_selection(all_inputs):

    D = len(all_inputs)
    setD = set(all_inputs)
    setS = [set() for i in range(D+1)]
    ellk = np.full(D+1,np.inf)

    setS[0] = set()

    for k in range(1,D+1):
        
        setA = [set() for i in range(D-k+1)]
        ellkappa = np.full(D-k+1,np.inf)

        for kappa, xp in enumerate(setD-setS[k-1]):
            setA[kappa] = setS[k-1].union({xp})
            theta0hat, theta1hat = train(setA[kappa],Dtrain)
            ellkappa[kappa] = assess(setA[kappa], 
                                     theta0hat,theta1hat, 
                                     Dvalidate)

        kappastar = ellkappa.argmin()
        setS[k] = setA[kappastar]
        ellk[k] = ellkappa[kappastar]

    kstar = ellk.argmin()
    Sstar = setS[kstar]
    theta0star, theta1star = train(Sstar, Dtrain)
    ellstar = assess(Sstar, theta0star, theta1star, Dtest)

    return ellk, ellstar, kstar

In [ ]:
# You can run forward stepwise selection on the collection of significant inputs

f_ellk, f_ellstar, f_kstar = forward_stepwise_selection(significant_inputs)

## 4.5 Backward stepwise selection

Implement backward stepwise selection.

In [ ]:
def backward_stepwise_selection(all_inputs):

    ...

    return ellk, ellstar, kstar

In [ ]:
# Use this cell to test your code

b_ellk, b_ellstar, b_kstar = backward_stepwise_selection(significant_inputs)

In [ ]:
grader.check("q4p5")

## Plot

The following plot shows the results of forward and backward stepwise feature selection for this regression problem. In each case, the star indicates the test error for the model with the smallest validation error.

In [ ]:
f_ellk, f_ellstar, f_kstar = forward_stepwise_selection(significant_inputs)
b_ellk, b_ellstar, b_kstar = backward_stepwise_selection(significant_inputs)
D = len(significant_inputs)

plt.figure(figsize=(10,5))

plt.plot(range(D+1),f_ellk,'o-',color='blue',label='forward',linewidth=3)
plt.plot([f_kstar,f_kstar],[f_ellk[f_kstar],f_ellstar],color='blue',linestyle='--',linewidth=2)
plt.plot(f_kstar,f_ellstar,'*',color='blue',markersize=22)

c = 'darkorange'
plt.plot(range(D+1),b_ellk,'o-',color=c,label='backward',linewidth=2)
plt.plot([b_kstar,b_kstar],[b_ellk[b_kstar],b_ellstar],color=c,linestyle=':',linewidth=2)
plt.plot(b_kstar,b_ellstar,'*',color=c,markersize=16)
plt.legend(fontsize=16)

plt.grid(linestyle=':')

# plt.ylim(340,480)
# plt.xlim(0,16)
plt.xticks(range(16),fontsize=16)
plt.xlabel('k',fontsize=16)
plt.ylabel('MSE',fontsize=16)

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

Make sure you submit the .zip file to Gradescope.

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False)